In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import pandas as pd
from tensorflow.keras import layers
from tqdm import tqdm
import matplotlib.pyplot as plt
import pathlib
import os

AUTOTUNE = tf.data.experimental.AUTOTUNE
tf.keras.backend.clear_session()  # For easy reset of notebook state.

tf.__version__
tf.executing_eagerly()


True

In [9]:

ds = tfds.load(name="imagenette", with_info=True, split=["train[:1000]","validation[:100]"])
ds_train=ds[0][0]
ds_test=ds[0][1]
assert isinstance(ds_train, tf.data.Dataset)

def normalize(x):
    y = {'image': tf.image.convert_image_dtype(x['image'], tf.float32), 'label': x['label']}
    y = (tf.image.resize(y['image'], (224,224)), y['label'])
    return y
    
ds_train = ds_train.map(lambda x: normalize(x))
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(100)
ds_train = ds_train.batch(4)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

ds_test = ds_test.map(
    normalize, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(4)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)


In [22]:
class CustomModel(tf.keras.Model):
    def __init__(self):
        super(CustomModel, self).__init__()
        self.layer1 = layers.Dense(512, activation='relu')
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.layer2 = layers.Dense(128, activation='relu')
        self.bn2 = tf.keras.layers.BatchNormalization()
        self.layer3 = layers.Dense(10, activation='softmax')
    def call(self,inputs, training=True):
        
        x = layers.Flatten()(inputs)
        #print(x)
        x = self.layer1(x)
        x = self.bn1(x, training = training)
        x = self.layer2(x)
        x = self.bn2(x, training = training)
        return self.layer3(x)

model = CustomModel()

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(0.001),
    metrics=['accuracy'],
)

model.fit(
    x=ds_train,
    #steps_per_epoch=1,
    epochs=15,
    validation_data=ds_test,
)

Tensor("keras_learning_phase:0", shape=(), dtype=bool)
Epoch 1/15
True
True
    250/Unknown - 95s 379ms/step - loss: 2.4189 - accuracy: 0.1800False
250/250 [==============================] - 96s 383ms/step - loss: 2.4189 - accuracy: 0.1800 - val_loss: 2.2808 - val_accuracy: 0.2800
Epoch 2/15
250/250 [==============================] - 93s 373ms/step - loss: 2.2275 - accuracy: 0.2190 - val_loss: 2.1355 - val_accuracy: 0.2300
Epoch 3/15
250/250 [==============================] - 79s 315ms/step - loss: 2.1158 - accuracy: 0.2500 - val_loss: 1.9986 - val_accuracy: 0.3400
Epoch 4/15
250/250 [==============================] - 79s 317ms/step - loss: 2.0837 - accuracy: 0.2820 - val_loss: 1.8596 - val_accuracy: 0.3300
Epoch 5/15
250/250 [==============================] - 81s 325ms/step - loss: 2.0066 - accuracy: 0.3200 - val_loss: 1.9556 - val_accuracy: 0.2700
Epoch 6/15
250/250 [==============================] - 75s 301ms/step - loss: 2.0198 - accuracy: 0.2910 - val_loss: 2.3515 - val_accuracy: 

In [ ]:
for example in ds_train.take(1):  # Only take a single example
    print(example['image'].shape)
    image, label = example["image"], example["label"]
    plt.imshow(image.numpy()[:, :, 0].astype(np.float32), cmap=plt.get_cmap("gray"))
    print("Label: %d" % label.numpy())

In [ ]:
ds_train = ds_train.repeat().shuffle(1024).batch(32)

# prefetch will enable the input pipeline to asynchronously fetch batches while
# your model is training.
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
tfds.show_examples(ds_train[1] ,ds_train[0]['train'])